# Classification models - Supervised Discretization
## Dataset: pima

Updated by: Sam
Update at: 13/10/2022 <br>
---
Discretization methods: DecisionTree (DT)<br>
Classification models: CNB, ID3, KNN-VDM
---
NOTE: 
NEED TO DO SMOTE for imbalance dataset before training
Long time for computation of Knn-VDM (Run this part last) <br>
Use Malina scripts for Knn, ID3 <br>
Use Sam scripts (with min_categories) for Naive Bayes <br>
Key Errors for training and decomposition KNN-VDM models => No metrics

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import six
import sys
sys.modules['sklearn.externals.six'] = six
import id3
from id3 import Id3Estimator
from id3 import export_graphviz
import wittgenstein as lw
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from pandas import read_csv
from pandas import set_option
import numpy as np
from numpy import arange
## EDA
from collections import Counter

# Knn-VDM 3
from vdm3 import ValueDifferenceMetric
from sklearn.neighbors import KNeighborsClassifier

# Pre-processing
from sklearn.preprocessing import OrdinalEncoder
# Cross validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score # 1 metric
from sklearn.model_selection import cross_validate # more than 1 metric
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [2]:
#read in data from ChiMerge discretization
pima_m2 = pd.read_csv('DT_small_discretized_pima.csv') # max_depth = 2
pima_m3 = pd.read_csv('DT_medium_discretized_pima.csv')  # max_depth = 3
pima_m4 = pd.read_csv('DT_large_discretized_pima.csv')  # max_depth = 4
pima_m5 = pd.read_csv('DT_verylarge_discretized_pima.csv')  # max_depth = 5

In [3]:
pima_m2.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,2,3,2,1,2,2,0,1
1,0,0,1,1,1,0,2,2,0
2,0,1,1,2,1,2,2,2,0
3,0,2,2,1,1,2,2,1,1
4,1,1,1,1,1,2,1,3,1


## Interval frequencies

In [4]:
num_list_m2 = list(pima_m2.drop('Outcome', axis=1).columns)
num_list_m3 = list(pima_m3.drop('Outcome', axis=1).columns)
num_list_m4 = list(pima_m4.drop('Outcome', axis=1).columns)
num_list_m5 = list(pima_m5.drop('Outcome', axis=1).columns)

In [5]:
print('Interval frequency for 6 Intervals')
for i in num_list_m2:
    print('Interval for ' + i)
    print(Counter(pima_m2[i]))

Interval frequency for 6 Intervals
Interval for Pregnancies
Counter({0: 488, 2: 165, 1: 111, 3: 4})
Interval for Glucose
Counter({0: 419, 1: 180, 3: 102, 2: 67})
Interval for BloodPressure
Counter({1: 479, 2: 154, 3: 134, 0: 1})
Interval for SkinThickness
Counter({2: 367, 1: 229, 0: 170, 3: 2})
Interval for Insulin
Counter({1: 375, 0: 230, 2: 157, 3: 6})
Interval for BMI
Counter({2: 560, 0: 170, 1: 24, 3: 14})
Interval for DiabetesPedigreeFunction
Counter({2: 308, 1: 208, 3: 135, 0: 117})
Interval for Age
Counter({3: 380, 2: 232, 0: 135, 1: 21})


In [6]:
print('Frequency for aus with 8 Intervals')
for i in num_list_m3:
    print('Interval for ' + i)
    print(Counter(pima_m3[i]))

Frequency for aus with 8 Intervals
Interval for Pregnancies
Counter({1: 250, 0: 238, 2: 111, 4: 111, 3: 54, 5: 4})
Interval for Glucose
Counter({1: 222, 0: 197, 3: 153, 7: 79, 4: 41, 2: 27, 5: 26, 6: 23})
Interval for BloodPressure
Counter({1: 404, 5: 122, 4: 84, 3: 75, 2: 70, 6: 12, 0: 1})
Interval for SkinThickness
Counter({3: 355, 2: 227, 1: 99, 0: 71, 5: 12, 6: 2, 4: 2})
Interval for Insulin
Counter({3: 374, 4: 155, 1: 117, 2: 113, 5: 4, 6: 3, 0: 2})
Interval for BMI
Counter({4: 411, 2: 149, 1: 126, 0: 53, 3: 15, 6: 11, 5: 3})
Interval for DiabetesPedigreeFunction
Counter({4: 260, 5: 152, 6: 121, 2: 61, 1: 56, 0: 56, 7: 48, 3: 14})
Interval for Age
Counter({6: 207, 7: 173, 4: 148, 3: 84, 2: 72, 1: 63, 5: 13, 0: 8})


In [7]:
print('Frequency for aus with 10 Intervals')
for i in num_list_m4:
    print('Interval for ' + i)
    print(Counter(pima_m4[i]))

Frequency for aus with 10 Intervals
Interval for Pregnancies
Counter({3: 200, 1: 135, 4: 111, 0: 103, 7: 83, 2: 50, 5: 44, 8: 28, 6: 10, 9: 4})
Interval for Glucose
Counter({1: 192, 5: 141, 7: 128, 2: 81, 15: 57, 10: 38, 9: 25, 12: 22, 11: 21, 6: 18, 14: 16, 3: 9, 4: 7, 13: 5, 8: 5, 0: 3})
Interval for BloodPressure
Counter({4: 358, 9: 111, 5: 51, 1: 46, 8: 45, 3: 40, 7: 39, 6: 30, 10: 24, 2: 11, 11: 9, 12: 3, 0: 1})
Interval for SkinThickness
Counter({5: 227, 6: 217, 4: 138, 2: 61, 1: 58, 3: 38, 0: 13, 8: 12, 9: 2, 7: 2})
Interval for Insulin
Counter({4: 374, 5: 144, 3: 109, 1: 108, 7: 11, 2: 9, 6: 7, 8: 4, 0: 2})
Interval for BMI
Counter({5: 340, 1: 123, 2: 103, 7: 71, 0: 54, 4: 46, 8: 15, 3: 13, 6: 3})
Interval for DiabetesPedigreeFunction
Counter({6: 247, 5: 148, 7: 103, 3: 54, 0: 45, 2: 34, 8: 30, 1: 29, 4: 27, 12: 18, 11: 18, 9: 7, 13: 4, 10: 4})
Interval for Age
Counter({10: 182, 11: 140, 5: 100, 2: 72, 1: 63, 6: 48, 3: 46, 4: 38, 9: 33, 8: 25, 7: 11, 0: 10})


In [8]:
print('Frequency for aus with 15 Intervals')
for i in num_list_m5:
    print('Interval for ' + i)
    print(Counter(pima_m5[i]))

Frequency for aus with 15 Intervals
Interval for Pregnancies
Counter({3: 143, 1: 135, 5: 111, 0: 103, 4: 57, 2: 50, 10: 45, 9: 38, 7: 35, 11: 28, 8: 10, 6: 9, 12: 4})
Interval for Glucose
Counter({2: 165, 10: 114, 7: 95, 5: 67, 4: 46, 0: 34, 19: 33, 20: 31, 13: 27, 11: 26, 15: 20, 3: 14, 12: 14, 1: 14, 8: 12, 18: 11, 14: 11, 17: 10, 6: 9, 16: 9, 9: 6})
Interval for BloodPressure
Counter({6: 350, 11: 75, 5: 50, 10: 45, 7: 40, 9: 39, 13: 36, 1: 35, 8: 30, 12: 13, 2: 11, 14: 11, 4: 11, 3: 8, 15: 6, 0: 5, 16: 3})
Interval for SkinThickness
Counter({9: 227, 11: 207, 8: 106, 5: 51, 1: 34, 10: 32, 4: 24, 3: 22, 7: 16, 0: 13, 13: 12, 2: 10, 6: 10, 14: 2, 12: 2})
Interval for Insulin
Counter({5: 374, 8: 127, 3: 70, 1: 56, 2: 52, 6: 39, 4: 17, 0: 10, 10: 9, 9: 7, 7: 7})
Interval for BMI
Counter({9: 329, 7: 89, 3: 64, 1: 59, 0: 54, 11: 46, 6: 36, 8: 25, 13: 15, 2: 14, 4: 11, 12: 10, 5: 10, 10: 6})
Interval for DiabetesPedigreeFunction
Counter({10: 128, 8: 120, 7: 119, 0: 59, 13: 57, 9: 46, 5: 32,

In [9]:
pima_m2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Pregnancies               768 non-null    int64
 1   Glucose                   768 non-null    int64
 2   BloodPressure             768 non-null    int64
 3   SkinThickness             768 non-null    int64
 4   Insulin                   768 non-null    int64
 5   BMI                       768 non-null    int64
 6   DiabetesPedigreeFunction  768 non-null    int64
 7   Age                       768 non-null    int64
 8   Outcome                   768 non-null    int64
dtypes: int64(9)
memory usage: 54.1 KB


# 1. Categorical Naive Bayes

## 1.1 CNB, DT, max_depth = 2

In [10]:
# make test & train split
X = pima_m2.drop('Outcome', axis=1)
y = pima_m2['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [11]:
# Check representation of class
print('Class representation - original: ', Counter(pima_m2['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [12]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [13]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [14]:
# SAM
# Count number of categories
n_categories = pima_m2.drop('Outcome', axis=1).nunique()

In [15]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 61
Accuracy: 0.68
Recall score :  0.6822916666666666
Precision score :  0.6822916666666666
F1 score :  0.6822916666666666
Classification report:
              precision    recall  f1-score   support

           0       0.88      0.59      0.71       125
           1       0.53      0.85      0.65        67

    accuracy                           0.68       192
   macro avg       0.70      0.72      0.68       192
weighted avg       0.76      0.68      0.69       192

Computation time:
0.012999773025512695


In [16]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.301
Average bias: 0.312
Average variance: 0.045
Sklearn 0-1 loss: 0.318


## 1.2 CNB, DT, max_depth = 3

In [22]:
# make test & train split
X = pima_m3.drop('Outcome', axis=1)
y = pima_m3['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [23]:
# Check representation of class
print('Class representation - original: ', Counter(pima_m3['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [24]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [25]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [26]:
# SAM
# Count number of categories
n_categories = pima_m3.drop('Outcome', axis=1).nunique()

In [27]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 46
Accuracy: 0.76
Recall score :  0.7604166666666666
Precision score :  0.7604166666666666
F1 score :  0.7604166666666666
Classification report:
              precision    recall  f1-score   support

           0       0.94      0.67      0.79       125
           1       0.60      0.93      0.73        67

    accuracy                           0.76       192
   macro avg       0.77      0.80      0.76       192
weighted avg       0.82      0.76      0.77       192

Computation time:
0.012999296188354492


In [28]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.251
Average bias: 0.250
Average variance: 0.049
Sklearn 0-1 loss: 0.240


## 1.3 CNB, DT, max_depth = 4

In [29]:
# make test & train split
X = pima_m4.drop('Outcome', axis=1)
y = pima_m4['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [30]:
# Check representation of class
print('Class representation - original: ', Counter(pima_m4['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [31]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [32]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [34]:
# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

Class representation - training data:  Counter({1: 375, 0: 375})


In [36]:
# SAM
# Count number of categories
n_categories = pima_m4.drop('Outcome', axis=1).nunique()

In [37]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 48
Accuracy: 0.75
Recall score :  0.75
Precision score :  0.75
F1 score :  0.75
Classification report:
              precision    recall  f1-score   support

           0       0.91      0.68      0.78       125
           1       0.60      0.88      0.71        67

    accuracy                           0.75       192
   macro avg       0.75      0.78      0.75       192
weighted avg       0.80      0.75      0.76       192

Computation time:
0.012998342514038086


In [38]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.256
Average bias: 0.250
Average variance: 0.061
Sklearn 0-1 loss: 0.250


## 1.4 CNB, DT, max_depth = 5

In [39]:
# make test & train split
X = pima_m5.drop('Outcome', axis=1)
y = pima_m5['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)


In [40]:
# Check representation of class
print('Class representation - original: ', Counter(pima_m5['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})


In [41]:
print(X_train.shape, X_test.shape)

(576, 8) (192, 8)


In [42]:
# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [43]:
# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

Class representation - training data:  Counter({1: 375, 0: 375})


In [44]:
# SAM
# Count number of categories
n_categories = pima_m5.drop('Outcome', axis=1).nunique()

In [45]:
import time
start = time.time() # For measuring time execution

cnb = CategoricalNB(min_categories = n_categories)
cnb.fit(X_train, y_train)

y_pred_cnb = cnb.predict(X_test)
y_prob_pred_cnb = cnb.predict_proba(X_test)

# how did our model perform?
count_misclassified = (y_test != y_pred_cnb).sum()
print("CategoricalNB")
print("=" * 25)
print('Misclassified samples: {}'.format(count_misclassified))
accuracy = accuracy_score(y_test, y_pred_cnb)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_cnb , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_cnb , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_cnb , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_cnb))

end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

CategoricalNB
Misclassified samples: 50
Accuracy: 0.74
Recall score :  0.7395833333333334
Precision score :  0.7395833333333334
F1 score :  0.7395833333333334
Classification report:
              precision    recall  f1-score   support

           0       0.89      0.69      0.77       125
           1       0.59      0.84      0.69        67

    accuracy                           0.74       192
   macro avg       0.74      0.76      0.73       192
weighted avg       0.78      0.74      0.75       192

Computation time:
0.01299905776977539


In [46]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        cnb, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_cnb))

Average expected loss: 0.251
Average bias: 0.255
Average variance: 0.069
Sklearn 0-1 loss: 0.260


# 2. Decision Tree ID3

## 2.1 ID3, DT, max_depth = 2

In [17]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m2.drop('Outcome', axis=1)
y = pima_m2['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m2['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m2.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [18]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.70
Recall score :  0.703125
Precision score :  0.703125
F1 score :  0.703125
Classification report:
              precision    recall  f1-score   support

           0       0.82      0.70      0.75       125
           1       0.56      0.72      0.63        67

    accuracy                           0.70       192
   macro avg       0.69      0.71      0.69       192
weighted avg       0.73      0.70      0.71       192

Computation time:
0.05203819274902344


In [19]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.323
Average bias: 0.281
Average variance: 0.181
Sklearn 0-1 loss: 0.297


## 2.2 ID3, DT, max_depth = 3

In [20]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m3.drop('Outcome', axis=1)
y = pima_m3['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m3['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m3.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [21]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.78
Recall score :  0.78125
Precision score :  0.78125
F1 score :  0.78125
Classification report:
              precision    recall  f1-score   support

           0       0.96      0.70      0.81       125
           1       0.62      0.94      0.75        67

    accuracy                           0.78       192
   macro avg       0.79      0.82      0.78       192
weighted avg       0.84      0.78      0.79       192

Computation time:
0.05827927589416504


In [22]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.289
Average bias: 0.240
Average variance: 0.175
Sklearn 0-1 loss: 0.219


## 2.3 ID3, DT, max_depth = 4

In [23]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m4.drop('Outcome', axis=1)
y = pima_m4['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m4['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m4.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [24]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.76
Recall score :  0.7604166666666666
Precision score :  0.7604166666666666
F1 score :  0.7604166666666666
Classification report:
              precision    recall  f1-score   support

           0       0.90      0.71      0.79       125
           1       0.61      0.85      0.71        67

    accuracy                           0.76       192
   macro avg       0.76      0.78      0.75       192
weighted avg       0.80      0.76      0.77       192

Computation time:
0.06502556800842285


In [25]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.286
Average bias: 0.198
Average variance: 0.192
Sklearn 0-1 loss: 0.240


## 2.4 ID3, DT, max_depth = 5

In [26]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m5.drop('Outcome', axis=1)
y = pima_m5['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m5['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m5.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [27]:
#time recording
import time
start = time.time() # For measuring time execution
#build estimator
estimator = Id3Estimator()
estimator = estimator.fit(X_train, y_train, check_input=True)
tree = export_graphviz(estimator.tree_, 'tree.dot', y)
#make predictions
y_pred_id3 = estimator.predict(X_test)
#_prob_pred_id3 = estimator.predict_proba(X_test)
#report performance
accuracy = accuracy_score(y_test, y_pred_id3)
print('Accuracy: {:.2f}'.format(accuracy))
print("=" * 25)
print("Recall score : ", recall_score(y_test, y_pred_id3 , average='micro'))
print("Precision score : ",precision_score(y_test, y_pred_id3 , average='micro'))
print("F1 score : ",f1_score(y_test, y_pred_id3 , average='micro'))
print("=" * 25)
print("Classification report:")
print(classification_report(y_test, y_pred_id3))
#stop time recoridng
end = time.time()
print("Computation time:")
print(end - start) # Total time execution for this sample

Accuracy: 0.71
Recall score :  0.7135416666666666
Precision score :  0.7135416666666666
F1 score :  0.7135416666666666
Classification report:
              precision    recall  f1-score   support

           0       0.80      0.74      0.77       125
           1       0.58      0.66      0.62        67

    accuracy                           0.71       192
   macro avg       0.69      0.70      0.69       192
weighted avg       0.72      0.71      0.72       192

Computation time:
0.09893083572387695


In [28]:
# calculate classification bias and variance
from sklearn.metrics import zero_one_loss
#This library is used to decompose bias and variance in our models
from mlxtend.evaluate import bias_variance_decomp
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import warnings
warnings.filterwarnings('ignore')
#We will load the Boston house dataset for our example
from sklearn.datasets import load_boston
from sklearn import metrics

# y_pred_cnb
# y_prob_pred_cnb

avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
        estimator, X_train.values, y_train.values, X_test.values, y_test.values, 
        loss='0-1_loss',
        random_seed=123)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_id3))

Average expected loss: 0.261
Average bias: 0.177
Average variance: 0.187
Sklearn 0-1 loss: 0.286


# 3. KNN classification

## 3.1 KNN-VDM, DT, max_depth = 2

In [29]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m2.drop('Outcome', axis=1)
y = pima_m2['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m2['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m2.drop('Outcome', axis=1).nunique()


Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [30]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

KeyError: 3.0

In [31]:
X_train.values

array([[0, 3, 1, ..., 0, 2, 3],
       [2, 3, 1, ..., 0, 2, 3],
       [1, 0, 1, ..., 2, 1, 2],
       ...,
       [2, 1, 2, ..., 2, 1, 3],
       [2, 3, 3, ..., 2, 3, 3],
       [0, 1, 3, ..., 2, 2, 0]], dtype=int64)

In [32]:
# # !!!! WARING: TIME CONSUMING

# # Bias and variance decomposition
# # Convert all dataframe to array first
# X_train = X_train.values
# X_test = X_test.values
# y_train = y_train.values
# y_test = y_test.values

# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train, y_train, X_test, y_test, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.2 KNN-VDM, DT, max_depth = 3

In [33]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m3.drop('Outcome', axis=1)
y = pima_m3['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m3['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m3.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [34]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

KeyError: 6.0

In [35]:
# # !!!! WARING: TIME CONSUMING

# # Bias and variance decomposition
# # Convert all dataframe to array first
# X_train = X_train.values
# X_test = X_test.values
# y_train = y_train.values
# y_test = y_test.values

# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train, y_train, X_test, y_test, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.3 KNN-VDM, DT, max_depth = 4

In [41]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m4.drop('Outcome', axis=1)
y = pima_m4['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m4['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m4.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [42]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

KeyError: 9.0

In [ ]:
# # !!!! WARING: TIME CONSUMING

# # Bias and variance decomposition
# # Convert all dataframe to array first
# X_train = X_train.values
# X_test = X_test.values
# y_train = y_train.values
# y_test = y_test.values

# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train, y_train, X_test, y_test, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))

## 3.4 KNN-VDM, DT, max_depth = 5

In [39]:
# Prepare data and apply SMOTE
# make test & train split
X = pima_m5.drop('Outcome', axis=1)
y = pima_m5['Outcome']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=30, stratify = y)

# Check representation of class
print('Class representation - original: ', Counter(pima_m5['Outcome'])) 
print('Class representation - training data: ', Counter(y_train)) 
print('Class representation - testing data: ', Counter(y_test)) 

print(X_train.shape, X_test.shape)

# SMOTE
#! pip install imblearn --user
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 22)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Check representation of class after SMOTE 
print('Class representation - training data: ', Counter(y_train)) 

# SAM
# Count number of categories
n_categories = pima_m5.drop('Outcome', axis=1).nunique()

Class representation - original:  Counter({0: 500, 1: 268})
Class representation - training data:  Counter({0: 375, 1: 201})
Class representation - testing data:  Counter({0: 125, 1: 67})
(576, 8) (192, 8)
Class representation - training data:  Counter({1: 375, 0: 375})


In [40]:
# Knn-VDM complete code
import time
start = time.time() # For measuring time execution

# specific the continuous columns index if any
vdm = ValueDifferenceMetric(X_train, y_train, continuous = None)
vdm.fit()
# Knn model, n_neigbour = 3, metrics = vdm
knn_vdm = KNeighborsClassifier(n_neighbors=3, metric=vdm.get_distance, algorithm='brute')
# Fit model
knn_vdm.fit(X_train, y_train)
# Testing
y_pred_knn = knn_vdm.predict(X_test)
knn_vdm.classes_
print(classification_report(y_test, y_pred_knn))

end = time.time()
print(f'Time for training model Knn-VDM: {end - start}.') # Total time execution

KeyError: 14.0

In [ ]:
# # !!!! WARING: TIME CONSUMING

# # Bias and variance decomposition
# # Convert all dataframe to array first
# X_train = X_train.values
# X_test = X_test.values
# y_train = y_train.values
# y_test = y_test.values

# avg_expected_loss, avg_bias, avg_var = bias_variance_decomp(
#         knn_vdm, X_train, y_train, X_test, y_test, 
#         loss='0-1_loss',
#         random_seed=123)

# print('Average expected loss: %.3f' % avg_expected_loss)
# print('Average bias: %.3f' % avg_bias)
# print('Average variance: %.3f' % avg_var)
# print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred_knn))